# IntegratedML applied to biomedical data, using PyODBC
This notebook demonstrates the following:
- Connecting to InterSystems IRIS via PyODBC connection
- Creating, Training and Executing (PREDICT() function) an IntegratedML machine learning model, applied to breast cancer tumor diagnoses
- INSERTING machine learning predictions into a new SQL table
- Executing a relatively complex SQL query containing IntegratedML PREDICT() and PROBABILITY() functions, and flexibly using the results to filter and sort the output

### ODBC and pyODBC Resources
Often, connecting to a database is more than half the battle when developing SQL-heavy applications, especially if you are not familiar with the tools, or more importantly the particular database system. If this is the case, and you are just getting started using PyODBC and InterSystems IRIS, this notebook and these resources below may help you get up to speed!

https://gettingstarted.intersystems.com/development-setup/odbc-connections/

https://irisdocs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=BNETODBC_support#BNETODBC_support_pyodbc

https://stackoverflow.com/questions/46405777/connect-docker-python-to-sql-server-with-pyodbc

https://stackoverflow.com/questions/44527452/cant-open-lib-odbc-driver-13-for-sql-server-sym-linking-issue

In [1]:
# make the notebook full screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1. Install system packages for ODBC

In [2]:
!apt-get update
!apt-get install gcc
!apt-get install -y tdsodbc unixodbc-dev
!apt install unixodbc-bin -y
!apt-get clean 

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [869 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1003 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]    
Get:9 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [9282 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [89.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:13 http://archive

Get:9 http://archive.ubuntu.com/ubuntu bionic/universe amd64 qtcore4-l10n all 4:4.8.7+dfsg-7ubuntu1 [617 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqtcore4 amd64 4:4.8.7+dfsg-7ubuntu1 [1552 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt4-xml amd64 4:4.8.7+dfsg-7ubuntu1 [96.1 kB]33m
Get:12 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqtdbus4 amd64 4:4.8.7+dfsg-7ubuntu1 [186 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt4-network amd64 4:4.8.7+dfsg-7ubuntu1 [562 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt4-script amd64 4:4.8.7+dfsg-7ubuntu1 [815 kB]
Get:15 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt4-sql amd64 4:4.8.7+dfsg-7ubuntu1 [98.5 kB]33m
Get:16 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt4-xmlpatterns amd64 4:4.8.7+dfsg-7ubuntu1 [1091 kB]
Get:17 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt4-declarative amd64 4:4.8.7+d

7Progress: [ 29%] [#################.........................................] 8Unpacking libqt4-script:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 30%] [#################.........................................] 8Selecting previously unselected package libqt4-sql:amd64.
Preparing to unpack .../14-libqt4-sql_4%3a4.8.7+dfsg-7ubuntu1_amd64.deb ...
7Progress: [ 32%] [##################........................................] 8Unpacking libqt4-sql:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 32%] [##################........................................] 87Progress: [ 33%] [###################.......................................] 8Selecting previously unselected package libqt4-xmlpatterns:amd64.
Preparing to unpack .../15-libqt4-xmlpatterns_4%3a4.8.7+dfsg-7ubuntu1_amd64.deb ...
Unpacking libqt4-xmlpatterns:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 35%] [####################......................................] 87Progress: [ 35%] [####################........

7Progress: [ 75%] [###########################################...............] 8Regenerating fonts cache... done.
Setting up libsm6:amd64 (2:1.2.2-1) ...
7Progress: [ 76%] [############################################..............] 87Progress: [ 77%] [############################################..............] 8Setting up libqt4-sql:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 79%] [#############################################.............] 8Setting up libxt6:amd64 (1:1.1.5-1) ...
7Progress: [ 79%] [##############################################............] 87Progress: [ 80%] [##############################################............] 8Setting up libqtdbus4:amd64 (4:4.8.7+dfsg-7ubuntu1) ...
7Progress: [ 82%] [###############################################...........] 8Setting up libaudio2:amd64 (1.9.4-6) ...
7Progress: [ 82%] [###############################################...........] 87Progress: [ 83%] [################################################.........

#### Use this command to troubleshoot a failed pyodbc installation:
!pip install --upgrade --global-option=build_ext --global-option="-I/usr/local/include"  --global-option="-L/usr/local/lib" pyodbc

In [3]:
!pip install pyodbc

     |████████████████████████████████| 266 kB 781 kB/s eta 0:00:01
  Created wheel for pyodbc: filename=pyodbc-4.0.30-cp36-cp36m-linux_x86_64.whl size=273428 sha256=9563b15d73734558f4d24d0724341ca82f0ca0486e3200ef68d2fee75a3f75ff
  Stored in directory: /root/.cache/pip/wheels/e3/3f/16/e11367542166d4f8a252c031ac3a4163d3b901b251ec71e905
Successfully built pyodbc


In [4]:
!rm /etc/odbcinst.ini
!rm /etc/odbc.ini

In [5]:
!ln -s /tf/odbcinst.ini /etc/odbcinst.ini
!ln -s /tf/odbc.ini /etc/odbc.ini

In [6]:
!cat /tf/odbcinst.ini

In [7]:
!cat /tf/odbc.ini

In [8]:
!odbcinst -j

unixODBC 2.3.4
DRIVERS............: /etc/odbcinst.ini
SYSTEM DATA SOURCES: /etc/odbc.ini
FILE DATA SOURCES..: /etc/ODBCDataSources
USER DATA SOURCES..: /root/.odbc.ini
SQLULEN Size.......: 8
SQLLEN Size........: 8
SQLSETPOSIROW Size.: 8


### 2. Verify you see "InterSystems ODBC35" in the drivers list

In [9]:
import pyodbc
print(pyodbc.drivers())

['InterSystems ODBC35']


### 3. Get an ODBC connection 

In [10]:
import pyodbc 
import time


#input("Hit any key to start")
dsn = 'IRIS QuickML demo via PyODBC'
server = 'irisimlsvr' #'192.168.99.101' 
port = '51773' #'9091'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### Ensure it read strings correctly.
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### 4. Get a cursor; start the timer

In [11]:
cursor = cnxn.cursor()
start= time.clock()

### 5. specify the training data, and give a model name

In [12]:
dataTable = 'SQLUser.BreastCancer'
dataTablePredict = 'Result02'
dataColumn =  'Diagnosis'
dataColumnPredict = "PredictedDiagnosis"
modelName = "bc" #chose a name - must be unique in server end

### 6. Train and predict

In [13]:
cursor.execute("CREATE MODEL %s PREDICTING (%s)  FROM %s" % (modelName, dataColumn, dataTable))
cursor.execute("TRAIN MODEL %s FROM %s" % (modelName, dataTable))
cursor.execute("Create Table %s (%s VARCHAR(100), %s VARCHAR(100))" % (dataTablePredict, dataColumnPredict, dataColumn))
cursor.execute("INSERT INTO %s  SELECT TOP 20 PREDICT(%s) AS %s, %s FROM %s" % (dataTablePredict, modelName, dataColumnPredict, dataColumn, dataTable)) 
cnxn.commit()

### 7. Show the predict result

In [14]:
import pandas as pd
from IPython.display import display

df1 = pd.read_sql("SELECT * from %s ORDER BY ID" % dataTablePredict, cnxn)
display(df1)

,PredictedDiagnosis,Diagnosis
0,M,M
1,M,M
2,M,M
3,M,M
4,M,M
5,M,M
6,M,M
7,M,M
8,M,M
9,M,M


### 8. Show a complicated query
IntegratedML function PREDICT() and PROBABILITY() can appear virtually anywhere in a SQL query, for maximal flexibility!
Below we are SELECTing columns as well as the result of the PROBABILITY function, and then filtering on the result of the PREDICT function. To top it off, ORDER BY is using the output of PROBSBILITY for sorting.

In [15]:
df2 = pd.read_sql("SELECT ID, PROBABILITY(bc FOR 'M') AS Probability, Diagnosis FROM %s \
                    WHERE MeanArea BETWEEN 300 AND 600 AND MeanRadius > 5 AND PREDICT(%s) = 'M' \
                    ORDER BY Probability" % (dataTable, modelName),cnxn)         
display(df2)

,ID,Probability,Diagnosis
0,82,0.504908,B
1,216,0.781883,M
2,147,0.892050,M
3,42,0.919181,M
4,436,0.963510,M
5,172,0.970692,M
6,40,0.971751,M
7,45,0.972587,M
8,6,0.972684,M
9,230,0.980062,M


### 9. Close and clean 

In [16]:
cnxn.close()
end= time.clock()
print ("Total elapsed time: ")
print (end-start)
#input("Hit any key to end")

Total elapsed time: 
0.6454539999999991
